# Glovo - Data Engineering Assessment


Welcome to the Glovo Data Engineer assessment. This assessment should take at most a few hours to complete.

When you are completed please download your notebook and send it to us. You can download it from the menu bar:

 ```File > Download as > Notebook```

This assessment will cover Python 3, SQL (PostgreSQL), and a few logic questions.

The Postgres database is pre-loaded with the tables as part of the Docker image, and it is publicly exposed so you can connect to it from your host computer if you wish (see the accompanying README.md file).

**Remember to download your completed notebook before terminating the Docker instance**

### What we value:

At Glovo we're looking for:

* A clean coding style

* Efficient solutions to the problems given

* Idiomatic use of Python

* Appropriate use of Python's built-in functions and standard libraries


## Python 3

### Q1. Explain the difference between a set and a list in Python. Give one use case for each, considering the computational cost of various operations.

The list is more similar to a classic array(such as C++ vectors), where the set is based on the Mathematical concept of a Set. 

The list has a focus on being able to store a series of elements in a specific order, and accessing to these elements is very fast if we know their index. This makes them ideal when we want to access every element. In Python specifically, lists are also fast when appending elements to the end, but slow when we want to insert elements at the front as we have to move all the elements that are after the insertion. 
One use case would be processing a series of n elements, where the processing is individual and independent. We can simply iterate through the list and each access will be done in constant time; the cost of going over the list will be linear.


The set has a focus on the basic operations defined for mathmatical sets: union, intersection, belonging, difference... This makes it ideal for cases where we precisely want to know the result of this operations and it is organized like a dictionary(hashmap) to check belonging faster, so we have no idea in which order the elements appear. An use case would be two groups of n elements and finding the common elements between the two groups. The way sets are organized, this would only take linear time on each set so the computational cost would be the sum of the size of both.

### Q2. Given the following list, count the number of each item, with the output in descending order - in a list of tuples

In [104]:
from collections import Counter

list_to_count = ['apple', 'banana', 'apple', 'apple', 'pear', 'banana', 'apple', 'banana']

### YOUR CODE HERE
result = sorted(Counter(list_to_count).items(),reverse=True)
print(result)


[('pear', 1), ('banana', 3), ('apple', 4)]


### Q3.  Call the given function repeatedly to produce a list of 100 integers between 1 and 10 (inclusive). Ignore any exceptions and any values outside of this range (these do not count towards the 100 integers). Sum this list of 100 integers.

In [105]:
import random


def badRandomNumber():
    x = random.randint(1, 15)
    if x > 12:
        raise ValueError
    return x

output = []
random.seed(123)  # Do not edit the seed

### YOUR CODE HERE
number=100
totalList=[None]*number #in this case the list is not needed and values 
while(number): #could be saved in a single variable
    try:
        x = badRandomNumber()
        if x>10:
            continue

        number-=1
        totalList[number]= x
    except:
        pass

result = sum(totalList)
print(result)



559


### Q4. Use the requests module to extract a list of the page IDs from the following API request.



In [106]:
import requests

request_url = """https://en.wikipedia.org/w/api.php?action=query&\
format=json&prop=extracts&titles=Stack%20Overflow%7CArch%20Linux%7cSuper%20Mario\
&utf8=1&exintro=1&exsectionformat=plain"""

### YOUR CODE HERE
page = requests.get(request_url)
id_list = [p for p in page.json()['query']['pages']]
print(id_list)

['674891', '21721040', '5249586']


### Q5. Convert the given list of datetimes from UTC to Europe/Madrid time

In [107]:
import datetime
import pytz

times = ['2017-07-01T11:00:00Z', '2018-01-02T05:00:00Z',
         '2017-09-01T11:00:00Z']

### YOUR CODE HERE
local_tz = pytz.timezone('Europe/Madrid')
fmt = '%Y-%m-%dT%H:%M:%SZ'
def date_to_local(date):
    temp = datetime.datetime.strptime(date,fmt)
    local_date = temp.replace(tzinfo=pytz.utc).astimezone(local_tz)
    return str(local_date.date()) +'T'+ str(local_date.timetz())+'Z'

result = [date_to_local(i) for i in times]
print(result)

['2017-07-01T13:00:00Z', '2018-01-02T06:00:00Z', '2017-09-01T13:00:00Z']


### Q6. Replace yes/no with boolean True/False in the following list of tuples

In [108]:
import string
import re

yesno_list = [(1, "yes"),
              (2, "Yes"),
              (3, "No"),
              (4, "no"),
              (5, "yes,"),
              (6, "no ")]

### YOUR CODE HERE
yesno_list = [(_, re.search('no',j,re.IGNORECASE) is None) for (_,j) in yesno_list ]
print(yesno_list)

[(1, True), (2, True), (3, False), (4, False), (5, True), (6, False)]


## SQL

Here we provide the connection details to the Postgres server and print the schema.

Run the cell below and continue to the SQL problems.


In [109]:
import psycopg2


conn = psycopg2.connect(
        dbname='dwh',
        host='127.0.0.1',
        port='5432',
        user='admin',
        password='admin',
    )
conn.autocommit = True
cursor = conn.cursor()

cursor.execute("""SELECT table_name
  FROM information_schema.tables
 WHERE table_schema='public'
   AND table_type='BASE TABLE';""")
tables = cursor.fetchall()
print("Tables: ")
for t in tables:
    print(" "+t[0])
    cursor.execute("""select column_name, data_type
from INFORMATION_SCHEMA.COLUMNS where table_name = '""" + t[0] + """';""")
    for c in cursor.fetchall():
        print("  " + c[0] + " :: " + c[1])
        


Tables: 
 orders
  creation_time :: timestamp without time zone
  order_id :: bigint
  user_id :: bigint
 scheduled_slots
  slot_time :: timestamp without time zone
  courier_id :: integer


### Q1. Get the order_id of the most recent order for each user in the orders table - give the output with the user_id in ascending order



In [110]:
cursor.execute("""
SELECT DISTINCT ON(user_id) user_id, order_id
FROM orders
ORDER BY  user_id,creation_time  DESC
""")#we could write ORDER BY  user_id ASC, creation_time DESC;
#but there is no need as ASC is the default


print('user_id, last_order_id')
print('\n'.join(', '.join(map(str, x)) for x in cursor.fetchall()))

user_id, last_order_id
289796, 4509163
904841, 5434211
925762, 4933672


### Q2. Calculate the average length of distinct sessions across all couriers from scheduled_slots

A distinct session is a one or more consecutive slots, that is, if the courier works 08-09am, 09-10am, and 10-11am together, that counts as one distinct session with a length of 3 hours.

Consecutive slots are slots done in one go, one after another, by the same courier - i.e. one hour on its own counts as one such slot, i.e. one distinct session of length 1 hour.

Two slots are consecutive (i.e. part of the same session) if they have exactly an hour's difference between their start_time's (as slots are per hour) and they have the same courier_id.

Note that we want the average length of the distinct sessions across all couriers, not the average of the average session length per courier.


In [111]:
cursor.execute("""
WITH temp AS (
	SELECT courier_id,
		slot_time - INTERVAL '1 hour' * ROW_NUMBER() OVER (PARTITION BY courier_id ORDER BY slot_time) AS sessions
	FROM scheduled_slots
	ORDER BY courier_id,slot_time)
SELECT CAST(COUNT(*) AS float) / COUNT(DISTINCT concat(courier_id, sessions))  
FROM temp""")#time spent on sessions/number of sessions

cursor.fetchall()

[(3.14285714285714,)]

## Logic

### Q1. Courier arrivals

Courier A arrives between 1pm and 3pm with uniform probability, Courier B arrives between 2pm and 4pm with uniform probability.

What is the probability that courier A arrives before courier B?

In [112]:
import numpy as np


np.random.seed(123)

### YOUR CODE HERE
#I like imagining these problems as a probability tree and expanding each subcase
#Case 1: A arrives before 2PM,0.5 times since we have uniform distribution (A is faster)
abefore2 = 0.5
#Case 2:A arrives after 2PM,0.5 times since we have uniform distribution
#Case 2.1:B arrives after 3PM, 0.5 times since we have uniform distribution (A is faster)
bafter3= 0.5*0.5
#Case 2.2:B arrives before 3PM, 0.5 times since we have uniform distribution 
#then since we have two independent variables with identical distribution
#the probability that one is higher than the other is chance, that is 1/2 ; 0.5 once again
between2and3= 0.5*0.5*0.5
A_is_faster = abefore2 + bafter3 + between2and3
print(A_is_faster)


0.875


### Q2. Two-headed coin in a bag

You have a bag of 1000 coins, in which one coin is two-headed. 

You take out one coin from the bag and flip it N times - obtaining heads every time.

Calculate the probability that you have taken the two-headed coin, for each $N$ in $0<=N<=20$ (i.e. at each flip what is the probability that you are holding the two-headed coin). 

Note that $N=0$ corresponds to no observations, i.e. your prior belief.

Give the probabilities to 3 decimal places.




In [113]:
### YOUR CODE HERE
for n in range(21):
    #a simple application of Bayes theorem
    probability = 0.001/( 0.001+ (0.999* 0.5**n))
    print(str(n)+ ':' + str(round(probability,3)))
    

0:0.001
1:0.002
2:0.004
3:0.008
4:0.016
5:0.031
6:0.06
7:0.114
8:0.204
9:0.339
10:0.506
11:0.672
12:0.804
13:0.891
14:0.943
15:0.97
16:0.985
17:0.992
18:0.996
19:0.998
20:0.999


### Assessment Completion

Thank you for completing the assessment.

Please download your completed notebook using the menu bar:

 ```File > Download as > Notebook```
 
 And return the .ipynb file to us.